In [1]:
import pandas as pd
import numpy as np
from typing import Tuple
from scipy.stats import mode
from sklearn.metrics import confusion_matrix
import seaborn as sns
path = "diabetes.csv"

In [18]:
def load_csv(path : str) -> Tuple[np.ndarray, np.ndarray]:
    np.random.seed(42)
    dataset = np.genfromtxt(path, delimiter = ',')
    np.random.shuffle(dataset)
    #pd.DataFrame.sample()
    x, y = dataset[:, :-1], dataset[:, -1]
    return x, y

In [19]:
load_csv(path)

(array([[  6.   ,  98.   ,  58.   , ...,  34.   ,   0.43 ,  43.   ],
        [ 13.   , 152.   ,  90.   , ...,  26.8  ,   0.731,  43.   ],
        [  2.   , 108.   ,  64.   , ...,  30.8  ,   0.158,  21.   ],
        ...,
        [  2.   , 146.   ,   0.   , ...,  27.5  ,   0.24 ,  28.   ],
        [  1.   ,  90.   ,  68.   , ...,  24.5  ,   1.138,  36.   ],
        [  1.   , 151.   ,  60.   , ...,  26.1  ,   0.179,  22.   ]]),
 array([ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,
         1.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,
         1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  0.,  0.,  0.,
         0.,  0.,  1.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  1.,
         0.,  0.,  1.,  0.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.,  1.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,
         1.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0